In [1]:
using Agents
using Random
using InteractiveDynamics
using CairoMakie

@agent tcell ContinuousAgent{2} begin
    activation_status::Int
    has_been_activated::Bool
end

@agent dcell ContinuousAgent{2} begin
        activation_status::Int
end

In [22]:
cells = Union{tcell, dcell}

function initialize_model(; 
    n_dcell = 30, 
    n_tcell = 30, 
    speed = 2,
    extent = (300,300),    
    visual_distance = 5.0,
    activation_status = 0,
     has_been_activated = false
    )

    space2d = ContinuousSpace(extent; spacing = 10,)
    
    model = ABM(cells, space2d ,rng = MersenneTwister(30),properties = Dict(:dt => 1.0); 
        scheduler = Schedulers.randomly)

    #scale the random number
    
    for _ in 1:n_tcell
        pos = Tuple(rand(model.rng, 2)).*300
        vel = sincos(2π * rand(model.rng)) .* speed 
        add_agent!(
            pos, tcell, model , vel, activation_status, has_been_activated
        )
    end
    
    for _ in 1:n_dcell
        pos = Tuple(rand(model.rng, 2)).*300
        vel = sincos(2π * rand(model.rng)) .* speed .* 0.001
        add_agent!(
            pos, dcell, model, vel, activation_status
        )
    end

    return model
end

initialize_model (generic function with 1 method)

In [29]:
model = initialize_model()

function agent_step!(cells,model;
        speed = 5,
       )
     
#     tcell.vel = sincos(2π * rand(model.rng)) .* speed 
#     dcell.vel = sincos(2π * rand(model.rng)) .* speed  .* 0.6
    
    if cells isa dcell
        cells.vel = sincos(2π * rand(model.rng)) .* speed .* 0.001
    else if cells isa tcell
        cells.vel = sincos(2π * rand(model.rng)) .* speed .*10

    end
    
        
     neighbor_ids = nearby_ids(cells,model,20)
    #neighbor of tcell 
    #neighbor of dcell
    N = 0
    
#     for id in neighbor_ids
#         N += 1
#         neighbor = model[id].pos
            
#         if euclidean_distance(tcell.pos, neighbor, model) < 10
#                 model[id].activated = true
    
#             end
#         end
#     end
    

    
    move_agent!(cells, model, model.dt)
end

CairoMakie.activate!()



┌ Warning: AgentType is not concrete. If your agent is parametrically typed, you're probably
│ seeing this warning because you gave `Agent` instead of `Agent{Float64}`
│ (for example) to this function. You can also create an instance of your agent
│ and pass it to this function. If you want to use `Union` types for mixed agent
│ models, you can silence this warning.
└ @ Agents /Users/kimchanghyeon/.julia/packages/Agents/QtIUJ/src/core/model.jl:299


LoadError: syntax: use "elseif" instead of "else if"

In [30]:
# offset(a) = a isa tcell ? (-0.1, -0.1*rand()) : (+0.1, +0.1*rand())
ashape(a) = a isa tcell ? :circle : :circle
asize(a) = a isa dcell ? 15 : 5
acolor(a) = a.activation_status > 0 ? :blue : :black
# ac(dcell) = :blue
# ac(tcell) = :black

acolor (generic function with 1 method)

In [31]:
abmvideo("tcell.mp4", model, agent_step!;
title = "immune cell", framerate  = 5, 
    frames = 200, as=asize, am = ashape, ac = acolor)